In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

from elm import ElmClient

In [ ]:
client = ElmClient("/dev/ttyUSB1")

In [ ]:
support = client.scan_pid_support()
print(support)
ecu_dfs = [
    pd.DataFrame({"ecu": ecu, "PID": ecu_support})
    for ecu, ecu_support in enumerate(support)
]
ecu_df = pd.concat(ecu_dfs).reset_index(drop=True)

In [ ]:
pid_table = pd.read_csv("../pid_table.csv")

In [ ]:
merged_df = ecu_df.merge(pid_table, on="PID", how="left")
merged_df.to_csv("../kia_pid_support.csv", index=False)
# merged_df

In [ ]:
support_set = set()
for _, group in merged_df.groupby("ecu"):
    if len(support_set) == 0:
        support_set = set(group["PID"])
    support_set.intersection(set(group["PID"]))
print(support_set)
all_support_df = merged_df[merged_df["PID"].isin(support_set)]
all_support_df = all_support_df[all_support_df["ecu"] == 0]
all_support_df.to_csv("kia_ecu_pid_all_support.csv", index=False)
# all_support_df

In [ ]:
ecu_unique = merged_df[~merged_df["PID"].isin(support_set)]
ecu_unique.to_csv("kia_ecu_pid_unique_support.csv", index=False)
ecu_unique